# Robotic Mapping with LiDAR Data

This project demonstrates LiDAR-based robotic mapping using affine transformations in 2D and 3D. The implementation explores translation, rotation, and scaling operations on point clouds and demonstrates how these can be combined to reconstruct environmental geometry from Cassie’s LiDAR data.

The objective is to transform raw LiDAR point clouds from the robot’s local frame into a global coordinate system, enabling map construction and environment visualization.

The project involves performing translation, rotation, and scaling operations on 2D and 3D point clouds using matrix representations. 
By applying these transformations to LiDAR scans, the robot’s local measurements are aligned in a unified world frame, forming a global map of the environment.


In [ ]:
using Plots

## Step 1 – 2D Transformations and Homogeneous Coordinates

In this step, simple geometric shapes are plotted and transformed using matrix operations to illustrate the concept of **affine transformations**. 
These transformations combine translation, rotation, and scaling within a single mathematical framework.

To represent these operations consistently, the coordinates are expressed in *homogeneous form*

In [ ]:
#=
    This function generates a 2D Plot
    Mandatory parameters are
                        xdata -> x coordinates
                        ydata -> y coordinates
                        color -> plot color
=#
function plot_2D(xdata::Vector{Float64}, ydata::Vector{Float64}, color; 
                 new=true, aspect_ratio=:equal, 
                 xlims = (-2,2), ylims = (-2,2), grid=true,
                 framestyle=:origin, legend=false, linewidth=3)
    fxn = new ? plot : plot! 
    fxn(xdata, ydata, seriescolor=color, aspect_ratio=aspect_ratio, xlims=xlims,
        ylims=ylims, grid=grid, framestyle=framestyle, legend=legend, 
        linewidth=linewidth)
end

#=
    This function generates a 2D Point Cloud Plot
    Mandatory parameters are
                        xdata -> x coordinates
                        ydata -> y coordinates
                        color -> pointcloud color
=#
function plot_2D_pointcloud(xdata::Vector{Float64}, ydata::Vector{Float64}, color; 
                new=true, seriestype=:scatter, markersize = 0.8, 
                markerstrokewidth = 0, dpi = 150, legend = false,
                markercolor  = cgrad(:rainbow, rev = true))
    fxn = new ? plot : plot!
    fxn(xdata, ydata, seriestype=seriestype, markersize=markersize, 
        markerstrokewidth=markerstrokewidth, dpi=dpi, 
        markercolor=markercolor, marker_z=color, legend=legend)
end

#=
    This function generates a 3D Point Cloud Plot
    Mandatory parameters are
                        xdata -> x coordinates
                        ydata -> y coordinates
                        zdata -> z coordinates
                        limits -> tuple of coordinate limits
                        color -> pointcloud color
=#
function plot_3D_pointcloud(xdata::Vector{Float64}, ydata::Vector{Float64}, zdata::Vector{Float64}, 
           limits, color::Vector{Float64};
            seriestype=:scatter3d, markersize=0.1, camera=(0,90), dpi=1080,
            markercolor=cgrad(:rainbow, rev=true), background_color=:black, 
            foreground_color=:black, legend=false, aspect_ratio=1, new=false)
    fxn = new ? plot : plot!
    fxn(xdata, ydata, zdata, xlims=limits[1], ylims=limits[2], zlims=limits[3], 
    seriestype=seriestype, markersize=markersize, camera=camera, dpi=dpi,
    markercolor=markercolor, marker_z=color, background_color=background_color,
    foreground_color=foreground_color, legend=legend, aspect_ratio=aspect_ratio,
    markerstrokewidth=0.001) 
end

#=
    This function plots the robot in 2D as a quiver
    Mandatory parameters are
                        xpos -> x coordinate of robot
                        ypos -> y coordinate of robot 
                        limits -> tuple of coordinate limits
                        gradient -> robot orientation (u,v)
=#
function plot_2D_robot(xpos::Vector{Float64}, ypos::Vector{Float64}, limits, gradient;
        seriestype=:quiver, projection="3d", arrow=(10,10), camera=(0,90), dpi=1080,
        background_color=:black, foreground_color=:black, legend=false, aspect_ratio=1, 
        seriescolor=:white, new=false)
    fxn = new ? plot : plot!
    fxn(xpos, ypos, seriescolor=seriescolor, seriestype=seriestype, projection=projection, 
        gradient=gradient, arrow=arrow, xlims=limits[1], ylims=limits[2], zlims=limits[3], 
        camera=camera, dpi=dpi, background_color=background_color, 
        foreground_color=foreground_color, legend=legend, aspect_ratio=aspect_ratio)
end

#=
    This function generates a 3D Point Cloud Plot
    Mandatory parameters are
                        xpos -> x coordinate of robot
                        ypos -> y coordinate of robot
                        zpos -> z coordinate of robot
=#
function plot_3D_robot(xpos::Vector{Float64}, ypos::Vector{Float64}, zpos::Vector{Float64}; 
                    markersize=4, seriestype=:scatter3d, seriescolor=:white, new=false)
    fxn = new ? plot : plot!
    fxn(xpos, ypos, zpos, seriescolor=seriescolor, markersize=markersize, seriestype=seriestype)
end    

In [ ]:
using Plots
gr()
# default(fmt = :png)

#=
    Defines the set of points forming a square for plotting.
    A fifth point is included to return to the starting vertex,
=#


# [x1 y1; x2 y2; ...; x5 y5]' 
points = [0. 0.; 0. 1.;1. 1.; 1. 0.; 0. 0.]';

# After the transpose, the array points looks like this
# [x1 x2 ... x5; 
#  y1 y2 ... y5]

# x coordinates extracted from the array
x = points[1,:]

# y coordinates extracted from the array
y = points[2,:]

# plotting the array of points in the same plot as a coordinate frame  
plot_2D(x, y, :green)

#### **Fundamental 2D Transformations**

Before working with LiDAR data, it is important to understand the three core geometric operations used to manipulate point coordinates in 2D:

**1. Translation**  which means moving the points by a constant offset  $t_x$ and $t_y$ : 

$\begin{bmatrix} x' \\ y' \end{bmatrix}=\begin{bmatrix} x \\ y \end{bmatrix} + \begin{bmatrix} t_x \\ t_y \end{bmatrix}$, where $t_x$ and $t_y$ are given constants. Here, x' is a new x point. It is not a transpose. The same goes for y'.

**2. Rotation**  
 which means rotating points counterclockwise by an angle $\theta$: 

 
$\begin{bmatrix} x'\\y'\end{bmatrix} = \begin{bmatrix}\cos{\theta}&-\sin{\theta} \\ \sin{\theta}&\cos{\theta}\end{bmatrix} \cdot \begin{bmatrix}x\\y\\ \end{bmatrix}$
where $\theta$ is the angle by which we wish to rotate a set of points in the counterclockwise direction, and finally

**3. Scaling**

Stretches or shrinks the coordinates by factors, potentially by diffrent amounts in the $x$ and $y$ directions.

Scaling looks like this $\begin{bmatrix} x'\\y' \end{bmatrix} = \begin{bmatrix}S_x&0\\0&S_y\\ \end{bmatrix} \cdot \begin{bmatrix}x\\y\\ \end{bmatrix}$




These transformations form the foundation of affine geometry. Later, they are combined to create unified transformation matrices that allow consistent handling of translation, rotation, and scaling in a single operation.




### **Step 1.A: Translation**

The first transformation applied to the square is **translation**. Each point of the square is shifted by 0.5 units along both the x-axis and y-axis to center the shape around the origin.
Mathematically, this operation is represented as:
 $\begin{bmatrix} x \\ y \end{bmatrix} = \begin{bmatrix} x \\ y \end{bmatrix} + \begin{bmatrix} t_x \\ t_y \end{bmatrix}$.

Here is $\begin{bmatrix} t_x \\ t_y \end{bmatrix} = \begin{bmatrix} -0.5 \\ -0.5 \end{bmatrix}$. The translated points are stored in an array named `points2` for subsequent transformations.


In [ ]:
# Define the translation vector t
t = [-0.5; -0.5]

# Recall that the points vector is organized like this, with each column
# of points containing an a x-y pair of coordinates that define corners 
# of the square plotted above.
#
# points = [x1 x2 ... x5; 
#           y1 y2 ... y5]

# Make a copy of the original points and call it points2
points2 = copy(points)
 
points2=points2 .+ t
x2 = points2[1,:];
y2 = points2[2,:];
 
plot_2D(x2, y2, :steelblue, new=false)

### **Step 1.B: Translation Using Homogeneous Coordinates**

To express translation as a matrix-vector multiplication, the points are represented in **homogeneous coordinates**. 
This allows translation, rotation, and scaling to share a unified mathematical form, simplifying computations on large LiDAR datasets.

A 2D point  $\begin{bmatrix} x \\ y \end{bmatrix}$ is converted to its homogeneous representation by appending a constant 1: $\begin{bmatrix} x \\ y \\ 1 \end{bmatrix}$  

The inverse operation simply removes the final entry, returning to Cartesian coordinates.

In homogeneous form, translation can be expressed as a single matrix multiplication:

$$
T =
\begin{bmatrix}
1 & 0 & t_x \\
0 & 1 & t_y \\
0 & 0 & 1
\end{bmatrix}
$$

Applying the transformation:

$$
T
\begin{bmatrix} x \\ y \\ 1 \end{bmatrix}
=
\begin{bmatrix} x + t_x \\ y + t_y \\ 1 \end{bmatrix}
$$

After removing the homogeneous term, the result in Cartesian space is: $\begin{bmatrix} x + t_x \\ y + t_y \end{bmatrix}$

This formulation enables translation to be handled identically to other affine transformations, making it particularly useful in robotic mapping and 3D data processing.

In [ ]:
# Define the translation matrix T
T = [1 0 -0.5;0 1 -0.5; 0 0 1];

# Make a copy of the original points
points3 = copy(points);

# Append 1 to each point to make homogeneous coordinates
points3 = [points3; ones(1,5)];

# Apply the translation to each of the points that form our square by
# multiplying each column of points3 by the matrix T and storing the result
# back in points3
points3 = T * points3 


# Extract x and y coordinates and throw away the number 1 at the end of the vector

x3 = points3[1,:];
y3 = points3[2,:];
 
plot_2D(x3, y3, :black, new=false, linewidth=5)

### **Step 1.C: Rotation in Homogeneous Coordinates**

The next transformation applies a **rotation** to the square using the 2D rotation matrix expressed in homogeneous coordinates:

$$
\begin{bmatrix}
x'\\
y'\\
1
\end{bmatrix}
=
\begin{bmatrix}
\cos{\theta} & -\sin{\theta} & 0 \\
\sin{\theta} &  \cos{\theta} & 0 \\
0 & 0 & 1
\end{bmatrix}
\begin{bmatrix}
x\\
y\\
1
\end{bmatrix}
$$

For this step, the square is rotated in the x–y plane by **45 degrees** ($\pi/4$ radians).  
The corresponding transformation matrix is constructed using the rotation angle \($\theta$ = $\pi/4$), and applied to all points of the square to obtain the rotated coordinates.  

In [ ]:
# Extract x and y coordinates and throw away the number 1 at the end
theta=pi/4

R=[cos(theta) -sin(theta) 0; sin(theta) cos(theta) 0; 0 0 1]
points3= R*points3
x3 = points3[1,:]
y3 = points3[2,:]


plot_2D(x3, y3, :red, new=false, linewidth=5)


### **Step 1.D: Scaling** 

The final transformation in this sequence is **scaling**, which adjusts the size of the square along the x- and y-axes.  
Scaling is a type of *non-rigid transformation*, as it changes the object’s dimensions without preserving distances.

The scaling operation in homogeneous coordinates is expressed as:

$$
\begin{bmatrix}
x'\\
y'\\
1
\end{bmatrix}
=
\begin{bmatrix}
S_x & 0 & 0 \\
0 & S_y & 0 \\
0 & 0 & 1
\end{bmatrix}
\begin{bmatrix}
x\\
y\\
1
\end{bmatrix}
$$

In this step, the rotated square is scaled using factors  
\( $S_x$ = 1.5 \) (horizontal stretching) and \( $S_y$ = 0.5 \) (vertical compression).  
The resulting transformation produces a rectangular shape with proportionally adjusted sides.

In [ ]:
# This portion of the the code implements the scaling Tranformation.   
Sx=1.5
Sy=0.5

S=[Sx 0 0;0 Sy 0;0 0 1]
points3=S*points3
# Extract x and y coordinates and throw away the last one
x3 = points3[1,:]
y3 = points3[2,:]

println("compare your plot to the one in the PDF of the project.")
 
plot_2D(x3, y3, :blue, new=false, linewidth=5)

### **Step 1.E: Combining Transformations: Affine Transformation**


All the transformations performed so far — translation, rotation, and scaling — can be expressed within a single framework known as an **Affine Transformation**.  

When working with large datasets, it is often necessary to apply several transformations sequentially. Managing these individually can be complex, but by expressing each operation in homogeneous coordinates, they can be combined into a single transformation matrix.

For example, consider a sequence consisting of:
1. Translation by \([-0.5, -0.5]\)
2. Rotation by 45° ($\pi/4$ radians) counterclockwise
3. Scaling by ( $S_x$ = 1.5 ) and ( $S_y$ = 0.5 \)

The composite transformation can be written as:

$$
T =
\begin{bmatrix}
1.5 & 0 & 0 \\
0 & 0.5 & 0 \\
0 & 0 & 1
\end{bmatrix}
\begin{bmatrix}
\cos(\pi/4) & -\sin(\pi/4) & 0 \\
\sin(\pi/4) &  \cos(\pi/4) & 0 \\
0 & 0 & 1
\end{bmatrix}
\begin{bmatrix}
1 & 0 & -0.5 \\
0 & 1 & -0.5 \\
0 & 0 & 1
\end{bmatrix}
$$

This combined matrix \(T\) represents the **overall affine transformation**, which can be applied directly to the set of points once they are expressed in homogeneous coordinates.

In the next step, the transformation matrix \(T\) is constructed as the product of the three individual matrices and applied to the original square to produce the final transformed shape.

In [ ]:
#Copying of the original points and appending a 1 to each point. 
points4 = [copy(points); ones(1,5)]

# points4 array contains the result of the affine transformation
T = S*R*T
points4 = T*points4

# Extract x and y coordinates and throw away the last one
x4 = points4[1,:]
y4 = points4[2,:]
 
plot_2D(x, y, :green)
plot_2D(x4, y4, :blue, new=false, linewidth=5)

# Step 2 – Working with 2D Point Clouds

This step introduces 2D point clouds as a precursor to the 3D LiDAR data analysis.  
A **point cloud** is a collection of spatial points that describe the geometry of an object or scene.  
For a 2D point cloud:
- The first two rows represent the \($x$\) and \($y$\) coordinates.
- The third row encodes an additional property, such as **intensity** (for LiDAR) or **color information** (for camera data).

In this dataset, the color values are simplified and normalized between 0 and 1, where 0 corresponds to red and 1 corresponds to blue.

The provided dataset contains a *distorted image* represented as a 2D point cloud.  
An affine transformation will be applied to correct this distortion and recover the readable structure of the original image.

The data is stored in the file **`question_image.csv`** within the `data/` directory.  
It can be loaded into an array using the `readdlm` function and visualized as a scatter plot to observe the initial distortion.

In [ ]:
using DelimitedFiles
using Plots
default(fmt = :png)

pointcloud = readdlm("data/question_image.csv",',')

# Extract the x and y coordinates from the first two rows of the data array.
point_data = pointcloud[1:2,:]

# The color (intensity) values are stored in the last row 
# and can be saved or processed separately.
println("Look at the size of the data you are manipulating! Yes, this is more interesting than the square.")
println(" ")
println("The vector pointcloud[3,:] has $(length(pointcloud[3,:])) elements. Oh my gosh!")
color_data = pointcloud[3,:]

In [ ]:
# Plot the 2D point cloud data with color values! 
plot_2D_pointcloud(point_data[1,:], point_data[2,:], color_data)

The affine transformation matrix used to correct the distorted point cloud is given by:

$$
T =
\begin{bmatrix}
-0.09239 & 0.038268 & 300 \\
-0.38268 & -0.923879 & 165 \\
0 & 0 & 1
\end{bmatrix}
$$

This transformation aligns the distorted points to form a readable image by applying a rotation, scaling, and translation in homogeneous coordinates.

In this step, the transformation \(T\) is applied to the distorted 2D point cloud, and the corrected image is visualized using a scatter plot.


In [ ]:
X = [copy(point_data); ones(1,size(point_data,2))]; # Create homogeneous 
                                                    # points
#=
Performs the following steps:
1. Construct the transformation matrix and store it in the variable `T`.
2. Apply `T` to all points to generate the transformed point cloud.
3. Save the transformed data directly into the array `X`.
=#
T = [-0.09239 0.038268 300;-0.38268 -0.923879 165;0 0 1]
n=size(X,2)

X=T*X

@show X


In [ ]:
# Note: Rendering this plot may take several seconds depending on the dataset size.
plot_2D_pointcloud(X[1,:], X[2,:], color_data)

After understanding 2D transformations, we extend these principles to 3D point clouds collected by Cassie’s LiDAR system.

# Step 3: 3D LiDAR Mapping and Visualization

This step extends the previous concepts to **3D LiDAR data** collected from the Cassie Blue robot. By the end of this section, a 3D point cloud map similar to the one shown below will be generated.

<img src="https://i.postimg.cc/qBFQ1ZZF/Task3.png" alt="Go Blue" width="900">

The dataset represents a region on the University of Michigan’s North Campus and is used here solely for educational and demonstration purposes.


Having explored matrix transformations in 2D, we now extend these concepts to **3D LiDAR data**.  
The following step involves visualizing the raw point cloud data collected by the Cassie Blue robot over a one-second interval.

Cassie’s LiDAR operates at **10 Hz**, capturing 10 individual scans per second.  
These scans have been combined into a single dataset, providing a large-scale example that demonstrates the computational challenges and efficiency of applying matrix-based transformations at scale.

The collected data is expressed in the **LiDAR coordinate frame**, which moves with the robot.  
A custom data-parsing function, `data_parser`, is provided to structure the information from multiple CSV files into usable arrays.  
Each time segment of the dataset is identified by an interval ID  
$id \in \{ 9, 10, \cdots, 13\}$, corresponding to specific time windows.

The `data_parser` function returns five arrays:

- **`pointcloud_data`**: the 3D coordinates (x, y, z) of the LiDAR points  
- **`Intensity_data`**: the measured LiDAR intensity values  
- **`R`**: the rotation matrix of the LiDAR frame  
- **`t`**: the translation vector  
- **`pose`**: the true position (xyz, column 1) and orientation (column 2) of the robot  

In the next section, these arrays will be used to visualize the raw point cloud and perform coordinate frame transformations.



In [ ]:
using LinearAlgebra
using DelimitedFiles
using Plots
default(fmt = :png)

# This is the data parser and helper functions block. 
function data_parser(id)
    # load point cloud from pointcloud.csv file
    points = readdlm(string("data/cassie_data/Frame_",
             string(id),"/pointcloud.csv"),',');
    pointcloud_data = points[1:3,:]; # xyz
    Intensity_data = points[4,:]; # intensity
    
    # load the different transformation from Transformations.csv file
    Transformations = readdlm(string("data/cassie_data/Frame_",
            string(id),"/Transformations.csv"),',');
    R = Transformations[:,1:3]; # rotation
    t = Transformations[:,4]; # translation
    
    # Load the robot’s position and orientation from Position.csv.
    pose = readdlm(string("data/cassie_data/Frame_",string(id),
            "/Position.csv"),',');
    return (pointcloud_data, Intensity_data, R, t, pose)
end

# Example call: parse the data for interval ID 10.
(pointcloud_data,Intensity_data,R,T,position) = data_parser(10)

# The following function applies a homogeneous transformation to a matrix of points.

#=  
  Apply a homogeneous transformation to the input points and 
  remove the appended ones after transformation.  
  This function demonstrates how to organize transformation code efficiently 
  for large-scale data processing.
=#

function affine_transform(points, H)
    # Create a copy of the points and append a row of ones (homogenization).
    transformed_p = [copy(points); ones(1,size(points,2))];
    
    # Apply the 3D transformation to all points.
    transformed_p = H * transformed_p
    
    # Return the de-homogenized points with the same dimensions as the input.
    return transformed_p[1:3,:];
end

#=  
  Utility function to extract the robot’s heading direction 
  for visualization purposes.  
  Used to plot an orientation arrow representing the robot’s pose.
=#

function robot_arrow(pose)
    position = pose[:,1]; # position
    (roll,pitch,yaw) = pose[:,2]; # orientation
    
    # Compute the rotation direction for visualization.
    u = [1*cos(yaw-pi/2)]
    v = [1*sin(yaw-pi/2)]
    return (position, u, v)
end

# Define viewing window limits and other plotting parameters.
window_size = 15
min_x = -window_size
min_y = -window_size
max_y = window_size
min_z = 0
max_z = 2*window_size
max_x = window_size;
limits = ((min_x, max_x), (min_y, max_y), (min_z, max_z));

### **Step 3.A: Parsing the 3D LiDAR Data**
This section demonstrates how the dataset is parsed and organized into structured arrays.  
The raw LiDAR data from the final collection interval (time = 13 seconds, `id = 13`) is read and processed using the `data_parser` function.

As discussed earlier, the function returns the following:
- **`pointcloud_data`**: 3D spatial coordinates of the LiDAR points  
- **`Intensity_data`**: corresponding LiDAR intensity values  
- **`R`**: rotation matrix  
- **`t`**: translation vector used to build the affine transformation  
- **`pose`**: the robot’s estimated position and orientation  

These outputs will be used to reconstruct and visualize the robot’s motion and surrounding environment.

In [ ]:
# Set the id number
id = 13

# Parse the data and save it into different arrays
(point_data, intensity_data, R, t, pose) = data_parser(id)

# Display the size of the point cloud to highlight the scale of the dataset.
# The dataset contains approximately 696,496 points — representing only one second
# of LiDAR data collected by Cassie Blue. 
# This demonstrates the computational scale and efficiency required for robotic data processing.
size(point_data) 

The LiDAR data is captured in the **sensor’s local coordinate frame**, which moves with the robot. As a result, each LiDAR scan corresponds to a different robot position and orientation.  
This situation is analogous to capturing multiple photographs while walking and then attempting to overlay them without alignment, resulting in visible displacement and distortion.

To illustrate the importance of consistent reference frames, multiple LiDAR scans are overlaid **without transformation to a common (world) frame**.  
This demonstrates how the lack of coordinate alignment leads to overlapping and misaligned point clouds.

> *Note: Generating the following plot may take several seconds due to the dataset size.*

In [ ]:
# Skipping half the data here to speed things up
start_index = 9
end_index  = 13

plot()
for id = start_index:2:end_index 
    
    # Save the data into different arrays
    (point_data, intensity_data, R, t, pose) = data_parser(id) 
    plot_3D_pointcloud(point_data[1,:], point_data[2,:], point_data[3,:], limits, intensity_data) 
end

display(plot!())

At this stage, multiple LiDAR scans have been collected, but their combined visualization appears disordered due to differences in position and orientation.  
To better understand the dataset, a single LiDAR scan is examined in isolation before integrating all frames.

As an initial step, the robot’s pose (position and orientation) is plotted in the **world coordinate frame** to establish a spatial reference for the LiDAR data.

In [ ]:
# Defines a function that generates an arrow to indicate the robot’s position 
# and orientation within a plot.
(point_data, intensity_data, R, t, pose) = data_parser(end_index) 
(pos, u, v) = robot_arrow(pose);

In [ ]:
# Plot the robot’s pose as a white arrow, followed by the LiDAR point cloud 
# on the same figure for spatial context.
plot()  
plot_2D_robot(pos[1,:], pos[2,:], limits, (u,v))

The white arrow in the plot represents the robot’s **position and orientation** within the world frame.  
With the pose established, the **raw LiDAR point cloud** is plotted in the same figure to visualize the spatial relationship between the sensor data and the robot’s location.

This comparison helps verify that the concentric rings of the LiDAR scan are geometrically consistent with Cassie’s pose and orientation.  

In [ ]:
# Plot the point cloud data.
id = end_index # = 13

# save the data into different arrays
(point_data, intensity_data, R, t, pose) = data_parser(id)

plot_3D_pointcloud(point_data[1,:], point_data[2,:], point_data[3,:], limits, intensity_data) 
display(plot!())

#### Transform

To correctly align the LiDAR data with the robot’s pose, the point cloud must be transformed from the **LiDAR frame** to the **world frame**.  
This is achieved by constructing an **affine transformation matrix** using the rotation matrix \( R \) and translation vector \( t \) obtained from the dataset.

An affine transformation in homogeneous coordinates is represented as:
\begin{bmatrix}R_{3 \times 3}& t_{3 \times 1} \\ 0_{1 \times 3} &1\\ \end{bmatrix}
where
- $R_{3 \times 3}$ is a 3 x 3 rotation matrix
- $t_{3 \times 1}$ is a 3 x 1 translation vector
- $0_{1 \times 3}$ is a 1 x 3 row vector of zeros
- and 1 is the number one.

After applying the affine transformation, the LiDAR point cloud is plotted together with the robot’s position estimate. The corrected data should now display **concentric LiDAR rings** centered around the robot’s pose.


In [ ]:
# Build the transformation from the rotation matrix and translation vector in our code
O=zeros(1,3)
H = [R t;O 1]


In [ ]:
# Use the verified transformation matrix constructed earlier.
# Apply the transformation and plot the results by calling the predefined function generalized from Step 1.
transformed_points = affine_transform(point_data, H)

# Visualize
plot() 
plot_3D_pointcloud(transformed_points[1,:], transformed_points[2,:], 
                    transformed_points[3,:], limits, intensity_data)

# Add robot pose to the plot 
plot_2D_robot(pos[1,:], pos[2,:], limits, (u,v))
display(plot!())
println("Displaying the transformed LiDAR point cloud aligned with the robot’s pose.")
println("The LiDAR rings are now correctly centered around the robot, indicating successful frame transformation.")



#### **Step 3.B: BUILDING A MAP and fusing different LiDAR scans!**

Having established how to transform LiDAR data from the sensor frame to the world frame, the next step is to **build a map** by combining data collected over multiple time intervals.  

A single one-second LiDAR scan provides only partial spatial coverage. To obtain a more complete view of the environment, scans captured at consecutive timestamps are fused together. In this demonstration, data from **five consecutive seconds** (time = 9 – 13 s) is used to generate a denser and more informative map.

Because the robot is continuously moving, each LiDAR frame must be transformed using the corresponding **rotation matrix** \( $R$ \) and **translation vector** \( $t$ \) to correctly align all scans in the world coordinate system. In a real robotic system, this transformation must be estimated continuously and in real-time to maintain an accurate global map.

This process illustrates a core principle in robotic perception:  
accurate motion compensation and frame alignment are essential for constructing coherent 3D representations from sequential sensor data.


In [ ]:
# Iterate over all LiDAR point clouds from time intervals 9 to 13.
# For each interval, apply the transformation and plot the results 
# on the same figure.
# Note: This process may take some time to execute due to data size.
start_index = 9
end_index  = 13
O=zeros(1,3)
final_points = [];
final_intensity = [];
for id = start_index : end_index 
    transformed_points = [];
    
#=
    Performs the following steps for each time interval:
    1. Parse the data for the current interval using the `data_parser` function
    and store it in arrays with the same variable names as before.
    2. Construct the transformation matrix for the current time interval 
    using the rotation matrix `R` and translation vector `t`.
    3. Apply the transformation to the point cloud and store the result 
    in an array named `transformed_points`.
=#

    (point_data, intensity_data, R, t, pose) = data_parser(id)
    H = [R t;O 1]
    transformed_points=affine_transform(point_data,H)
    

    if(id==start_index)
        final_points = copy(transformed_points)
        final_intensity = intensity_data'
    else
        final_points = [final_points transformed_points]
        final_intensity = [final_intensity intensity_data']
        println("Percent remaining is $(100*(end_index-id)/(end_index-start_index)).")
    end
    
end
@show final_points[:,1:5]

In [ ]:
# This process may take some time to execute
plot()
# Update the plot 
plot_3D_pointcloud(final_points[1,:], final_points[2,:], final_points[3,:], limits, final_intensity')
display(plot!())

### Generating a Map-Building Animation

With the 3D LiDAR map successfully reconstructed, the next step is to visualize the mapping process as it unfolds over time. By iterating over all time intervals, individual LiDAR scans can be sequentially added to the global map to create an animation of the mapping process.

The same transformation procedure used previously is applied within a loop over all frames, each frame displays both the accumulated LiDAR data and the robot’s position, represented as a white marker.

The resulting animation provides an intuitive visualization of how the environment is incrementally mapped as the robot moves through it.

In [ ]:
start_index = 9
end_index  = 13
Plots.plot()
# Build an animation!

anim = @animate for id = start_index:end_index 
    transformed_points = [];
    (point_data, intensity_data, R, t, pose) = data_parser(id)
    H = [R t;O 1]
    transformed_points=affine_transform(point_data,H)    
    plot_3D_pointcloud(transformed_points[1,:], transformed_points[2,:], transformed_points[3,:], 
            limits, intensity_data)
    
    if(id!=start_index)
        # Plot your previous position with a black dot everytime except the first
        (point_data, intensity_data, R, t, prev_pose) = data_parser(id-1); 
        plot_3D_robot([prev_pose[1,1]],[prev_pose[2,1]],[prev_pose[3,1]], seriescolor=:black)
    end
     
    plot_3D_robot([pose[1,1]],[pose[2,1]],[pose[3,1]],seriescolor=:white)
    println("Percent done is $(100*(1+id-start_index)/(1+end_index-start_index)) %")

    prev_pose = copy(pose);
end
println("Generating the gif animation...")
println("The white dot represents Cassie’s position within the reconstructed map,")
println("while the surrounding environment remains stationary.")
gif(anim, "stairs_walking_five_seconds_zoomed_thirty.gif", fps = 1)

## Results and Discussion

The LiDAR transformation pipeline successfully reconstructed Cassie’s environment by transforming data from the robot’s local sensor frame into the global world frame. The generated plots and GIF illustrate consistent alignment of LiDAR rings, confirming correct application of translation, rotation, and scaling transformations.


## Conclusion

In this project, we explored the use of **matrix transformations and LiDAR data** for robotic mapping starting from basic 2D affine transformations, the workflow progressed to **3D LiDAR data processing**, frame alignment, and **multi-frame map construction**.  

The final result demonstrated how sequential LiDAR scans can be transformed and fused to reconstruct a coherent spatial map as the robot moves.

This workflow highlights key concepts in robotic perception, including coordinate transformations, sensor fusion, and motion compensation.



## Acknowledgment

The LiDAR dataset and instructional framework were provided by **University of Michigan Robotics Institute (ROB 101 course materials)**. The implementation, refinement, and documentation presented here were independently completed for learning and demonstration purposes.